# Amazon Bedrock AgentCore 런타임에서 MCP 서버 호스팅

## 개요

이 튜토리얼에서는 Amazon Bedrock AgentCore 런타임에서 MCP(모델 컨텍스트 프로토콜) 서버를 호스팅하는 방법을 알아봅니다. Amazon Bedrock AgentCore Python SDK를 사용하여 MCP 도구를 Amazon Bedrock AgentCore와 호환되는 MCP 서버로 래핑합니다.

Amazon Bedrock AgentCore Python SDK는 MCP 서버 구현 세부 정보를 처리하므로 사용자는 도구의 핵심 기능에 집중할 수 있습니다. 또한 코드를 AgentCore 표준 MCP 프로토콜 계약으로 변환하여 직접 통신할 수 있도록 합니다.

### 튜토리얼 세부 정보

| 정보 | 세부 정보 |
|:--------------------|:----------------------------------------------------------|
| 튜토리얼 유형 | 호스팅 도구 |
| 도구 유형 | MCP 서버 |
| 튜토리얼 구성 요소 | AgentCore 런타임에서 MCP 서버 호스팅 |
| 튜토리얼 수직 | 교차 수직 |
| 예제 복잡성 | 쉬움 |
| 사용된 SDK | Amazon BedrockAgentCore Python SDK 및 MCP |

### 튜토리얼 아키텍처

이 튜토리얼에서는 AgentCore 런타임에 MCP 서버를 배포하는 방법을 설명합니다.

시연을 위해 `add_numbers`, `multiply_numbers`, `greet_user`라는 세 가지 도구를 갖춘 간단한 MCP 서버를 사용합니다.

<div style="text-align:left">
<img src="images/hosting_mcp_server.png" width="60%"/>
</div>

### 튜토리얼 주요 기능

* 사용자 지정 도구를 사용하여 MCP 서버 생성
* 로컬에서 MCP 서버 테스트
* Amazon Bedrock AgentCore 런타임에 MCP 서버 호스팅
* 인증을 통해 배포된 MCP 서버 호출

## 사전 요구 사항

이 튜토리얼을 실행하려면 다음이 필요합니다.
* Python 3.10 이상
* AWS 자격 증명 구성
* Amazon Bedrock AgentCore SDK
* MCP(Model Context Protocol) 라이브러리
* Docker 실행

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

## Understanding MCP (Model Context Protocol)

MCP is a protocol that allows AI models to securely access external data and tools. Key concepts:

* **Tools**: Functions that the AI can call to perform actions
* **Streamable HTTP**: Transport protocol used by AgentCore Runtime
* **Session Isolation**: Each client gets isolated sessions via `Mcp-Session-Id` header
* **Stateless Operation**: Servers must support stateless operation for scalability

AgentCore Runtime expects MCP servers to be hosted on `0.0.0.0:8000/mcp` as the default path.

### Project Structure

Let's set up our project with the proper structure:

```
mcp_server_project/
├── mcp_server.py              # Main MCP server code
├── my_mcp_client.py          # Local testing client
├── my_mcp_client_remote.py   # Remote testing client
├── requirements.txt          # Dependencies
└── __init__.py              # Python package marker
```

## MCP 서버 만들기

이제 세 가지 간단한 도구를 사용하여 MCP 서버를 만들어 보겠습니다. 이 서버는 AgentCore 런타임 호환성을 위해 `stateless_http=True`를 설정한 FastMCP를 사용합니다.

In [ ]:
%%writefile mcp_server.py
from mcp.server.fastmcp import FastMCP
from starlette.responses import JSONResponse

mcp = FastMCP(host="0.0.0.0", stateless_http=True)

@mcp.tool()
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together"""
    return a + b

@mcp.tool()
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers together"""
    return a * b

@mcp.tool()
def greet_user(name: str) -> str:
    """Greet a user by name"""
    return f"Hello, {name}! Nice to meet you."

if __name__ == "__main__":
    mcp.run(transport="streamable-http")

### 이 코드의 기능

* **FastMCP**: 도구를 호스팅할 수 있는 MCP 서버를 생성합니다.
* **@mcp.tool()**: Python 함수를 MCP 도구로 변환하는 데코레이터입니다.
* **stateless_http=True**: AgentCore 런타임 호환성을 위해 필요합니다.
* **도구**: 다양한 유형의 작업을 보여주는 세 가지 간단한 도구입니다.

## 로컬 테스트 클라이언트 생성

AgentCore 런타임에 배포하기 전에 MCP 서버를 로컬에서 테스트할 클라이언트를 생성해 보겠습니다.

In [ ]:
%%writefile my_mcp_client.py
import asyncio
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    mcp_url = "http://localhost:8000/mcp"
    headers = {}

    async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
        read_stream,
        write_stream,
        _,
    ):
        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            tool_result = await session.list_tools()
            print("Available tools:")
            for tool in tool_result.tools:
                print(f"  - {tool.name}: {tool.description}")

if __name__ == "__main__":
    asyncio.run(main())

### 로컬 테스트

MCP 서버를 로컬에서 테스트하려면 다음을 수행하세요.

1. **터미널 1**: MCP 서버 시작
```bash
python mcp_server.py
```

2. **터미널 2**: 테스트 클라이언트 실행
```bash
python my_mcp_client.py
```

출력 결과에 세 가지 도구가 표시됩니다.

## Amazon Cognito에서 인증 설정하기

AgentCore 런타임에는 인증이 필요합니다. 배포된 MCP 서버에 액세스하기 위한 JWT 토큰을 제공하기 위해 Amazon Cognito를 사용합니다.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import setup_cognito_user_pool

In [ ]:
print("Setting up Amazon Cognito user pool...")
cognito_config = setup_cognito_user_pool()
print("Cognito setup completed ✓")
print(f"User Pool ID: {cognito_config.get('user_pool_id', 'N/A')}")
print(f"Client ID: {cognito_config.get('client_id', 'N/A')}")

## AgentCore 런타임 배포 구성

다음으로, 시작 툴킷을 사용하여 진입점, 방금 생성한 실행 역할, 그리고 요구 사항 파일을 사용하여 AgentCore 런타임 배포를 구성합니다. 또한 시작 툴킷이 실행 시 Amazon ECR 저장소를 자동으로 생성하도록 구성합니다.

구성 단계에서는 애플리케이션 코드를 기반으로 Docker 파일이 생성됩니다.

<div style="text-align:left">
<img src="images/configure.png" width="60%"/>
</div>

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time

boto_session = Session()
region = boto_session.region_name
print(f"Using AWS region: {region}")

required_files = ['mcp_server.py', 'requirements.txt']
for file in required_files:
    if not os.path.exists(file):
        raise FileNotFoundError(f"Required file {file} not found")
print("All required files found ✓")

agentcore_runtime = Runtime()

auth_config = {
    "customJWTAuthorizer": {
        "allowedClients": [
            cognito_config['client_id']
        ],
        "discoveryUrl": cognito_config['discovery_url'],
    }
}

print("Configuring AgentCore Runtime...")
response = agentcore_runtime.configure(
    entrypoint="mcp_server.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    authorizer_configuration=auth_config,
    protocol="MCP",
    agent_name="mcp_server_agentcore"
)
print("Configuration completed ✓")

## AgentCore 런타임에 MCP 서버 실행

이제 Docker 파일이 생성되었으므로 AgentCore 런타임에 MCP 서버를 실행해 보겠습니다. 이렇게 하면 Amazon ECR 저장소와 AgentCore 런타임이 생성됩니다.

<div style="text-align:left">
<img src="images/launch.png" width="85%"/>
</div>

In [ ]:
print("Launching MCP server to AgentCore Runtime...")
print("This may take several minutes...")
launch_result = agentcore_runtime.launch()
print("Launch completed ✓")
print(f"Agent ARN: {launch_result.agent_arn}")
print(f"Agent ID: {launch_result.agent_id}")

## AgentCore 런타임 상태 확인

이제 AgentCore 런타임을 배포했으니, 배포 상태를 확인하고 준비가 될 때까지 기다려 보겠습니다.

In [ ]:
print("Checking AgentCore Runtime status...")
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
print(f"Initial status: {status}")

end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    print(f"Status: {status} - waiting...")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']

if status == 'READY':
    print("✓ AgentCore Runtime is READY!")
else:
    print(f"⚠ AgentCore Runtime status: {status}")
    
print(f"Final status: {status}")

## 원격 액세스를 위한 구성 저장

배포된 MCP 서버를 호출하기 전에, 에이전트 ARN과 Cognito 구성을 AWS Systems Manager Parameter Store와 AWS Secrets Manager에 저장하여 쉽게 검색할 수 있도록 해보겠습니다.

In [ ]:
import boto3
import json

ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    cognito_credentials_response = secrets_client.create_secret(
        Name='mcp_server/cognito/credentials',
        Description='Cognito credentials for MCP server',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials stored in Secrets Manager")
except secrets_client.exceptions.ResourceExistsException:
    secrets_client.update_secret(
        SecretId='mcp_server/cognito/credentials',
        SecretString=json.dumps(cognito_config)
    )
    print("✓ Cognito credentials updated in Secrets Manager")

agent_arn_response = ssm_client.put_parameter(
    Name='/mcp_server/runtime/agent_arn',
    Value=launch_result.agent_arn,
    Type='String',
    Description='Agent ARN for MCP server',
    Overwrite=True
)
print("✓ Agent ARN stored in Parameter Store")

print("\nConfiguration stored successfully!")
print(f"Agent ARN: {launch_result.agent_arn}")

## 원격 테스트 클라이언트 생성

이제 배포된 MCP 서버를 테스트할 클라이언트를 생성해 보겠습니다. 이 클라이언트는 AWS에서 필요한 자격 증명을 가져와 배포된 서버에 연결합니다.

In [ ]:
%%writefile my_mcp_client_remote.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    if not agent_arn or not bearer_token:
        print("Error: AGENT_ARN or BEARER_TOKEN not retrieved properly")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")
    print("Headers configured ✓")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}")
                    print(f"   Description: {tool.description}")
                    if hasattr(tool, 'inputSchema') and tool.inputSchema:
                        properties = tool.inputSchema.get('properties', {})
                        if properties:
                            print(f"   Parameters: {list(properties.keys())}")
                    print()
                
                print(f"✅ Successfully connected to MCP server!")
                print(f"Found {len(tool_result.tools)} tools available.")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## 배포된 MCP 서버 테스트

원격 클라이언트를 사용하여 배포된 MCP 서버를 테스트해 보겠습니다.

In [ ]:
print("Testing deployed MCP server...")
print("=" * 50)
!python my_mcp_client_remote.py

## MCP 도구 원격 호출

이제 도구를 나열할 뿐만 아니라 도구를 호출하여 전체 MCP 기능을 시연하는 향상된 클라이언트를 만들어 보겠습니다.

In [ ]:
%%writefile invoke_mcp_tools.py
import asyncio
import boto3
import json
import sys
from boto3.session import Session
from datetime import timedelta

from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client

async def main():
    boto_session = Session()
    region = boto_session.region_name
    
    print(f"Using AWS region: {region}")
    
    try:
        ssm_client = boto3.client('ssm', region_name=region)
        agent_arn_response = ssm_client.get_parameter(Name='/mcp_server/runtime/agent_arn')
        agent_arn = agent_arn_response['Parameter']['Value']
        print(f"Retrieved Agent ARN: {agent_arn}")

        secrets_client = boto3.client('secretsmanager', region_name=region)
        response = secrets_client.get_secret_value(SecretId='mcp_server/cognito/credentials')
        secret_value = response['SecretString']
        parsed_secret = json.loads(secret_value)
        bearer_token = parsed_secret['bearer_token']
        print("✓ Retrieved bearer token from Secrets Manager")
        
    except Exception as e:
        print(f"Error retrieving credentials: {e}")
        sys.exit(1)
    
    encoded_arn = agent_arn.replace(':', '%3A').replace('/', '%2F')
    mcp_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{encoded_arn}/invocations?qualifier=DEFAULT"
    headers = {
        "authorization": f"Bearer {bearer_token}",
        "Content-Type": "application/json"
    }
    
    print(f"\nConnecting to: {mcp_url}")

    try:
        async with streamablehttp_client(mcp_url, headers, timeout=timedelta(seconds=120), terminate_on_close=False) as (
            read_stream,
            write_stream,
            _,
        ):
            async with ClientSession(read_stream, write_stream) as session:
                print("\n🔄 Initializing MCP session...")
                await session.initialize()
                print("✓ MCP session initialized")
                
                print("\n🔄 Listing available tools...")
                tool_result = await session.list_tools()
                
                print("\n📋 Available MCP Tools:")
                print("=" * 50)
                for tool in tool_result.tools:
                    print(f"🔧 {tool.name}: {tool.description}")
                
                print("\n🧪 Testing MCP Tools:")
                print("=" * 50)
                
                try:
                    print("\n➕ Testing add_numbers(5, 3)...")
                    add_result = await session.call_tool(
                        name="add_numbers",
                        arguments={"a": 5, "b": 3}
                    )
                    print(f"   Result: {add_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n✖️  Testing multiply_numbers(4, 7)...")
                    multiply_result = await session.call_tool(
                        name="multiply_numbers",
                        arguments={"a": 4, "b": 7}
                    )
                    print(f"   Result: {multiply_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                try:
                    print("\n👋 Testing greet_user('Alice')...")
                    greet_result = await session.call_tool(
                        name="greet_user",
                        arguments={"name": "Alice"}
                    )
                    print(f"   Result: {greet_result.content[0].text}")
                except Exception as e:
                    print(f"   Error: {e}")
                
                print("\n✅ MCP tool testing completed!")
                
    except Exception as e:
        print(f"❌ Error connecting to MCP server: {e}")
        sys.exit(1)

if __name__ == "__main__":
    asyncio.run(main())

## 테스트 도구 호출

MCP 도구를 실제로 호출하여 테스트해 보겠습니다.

In [ ]:
print("Testing MCP tool invocation...")
print("=" * 50)
!python invoke_mcp_tools.py

## 다음 단계

MCP 서버를 AgentCore 런타임에 성공적으로 배포했으므로 이제 다음을 수행할 수 있습니다.

1. **도구 추가**: 추가 도구를 사용하여 MCP 서버 확장
2. **사용자 지정 인증**: 사용자 지정 JWT 권한 부여자 구현
3. **통합**: 다른 AgentCore 서비스와 통합

## 정리(선택 사항)

이 튜토리얼에서 생성된 리소스를 정리하려면 다음 셀을 실행하세요.

In [ ]:
import boto3

print("🗑️  Starting cleanup process...")

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)
ecr_client = boto3.client('ecr', region_name=region)
ssm_client = boto3.client('ssm', region_name=region)
secrets_client = boto3.client('secretsmanager', region_name=region)

try:
    print("Deleting AgentCore Runtime...")
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )
    print("✓ AgentCore Runtime deletion initiated")

    print("Deleting ECR repository...")
    ecr_repo_name = launch_result.ecr_uri.split('/')[1]
    ecr_client.delete_repository(
        repositoryName=ecr_repo_name,
        force=True
    )
    print("✓ ECR repository deleted")

    try:
        ssm_client.delete_parameter(Name='/mcp_server/runtime/agent_arn')
        print("✓ Parameter Store parameter deleted")
    except ssm_client.exceptions.ParameterNotFound:
        print("ℹ️  Parameter Store parameter not found")
    try:
        secrets_client.delete_secret(
            SecretId='mcp_server/cognito/credentials',
            ForceDeleteWithoutRecovery=True
        )
        print("✓ Secrets Manager secret deleted")
    except secrets_client.exceptions.ResourceNotFoundException:
        print("ℹ️  Secrets Manager secret not found")

    print("\n✅ Cleanup completed successfully!")
    
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

# 🎉 축하합니다!

다음 작업을 성공적으로 완료했습니다.

✅ 사용자 지정 도구를 사용하여 **MCP 서버 생성**
✅ MCP 클라이언트를 사용하여 **로컬에서 테스트**
✅ Amazon Cognito를 사용하여 **인증 설정**
✅ AgentCore Runtime을 사용하여 **AWS에 배포**
✅ 적절한 인증을 사용하여 **원격으로 호출**
✅ **MCP 개념** 및 모범 사례 학습

이제 MCP 서버가 Amazon Bedrock AgentCore Runtime에서 실행 중이며 프로덕션 환경에서 사용할 준비가 되었습니다!

## 요약

이 튜토리얼에서는 다음 방법을 학습했습니다.
- FastMCP를 사용하여 MCP 서버 구축
- AgentCore와의 호환성을 위해 상태 비저장 HTTP 전송 구성
- Amazon Cognito를 사용하여 JWT 인증 설정
- AWS에 MCP 서버 배포 및 관리
- 로컬 및 원격 테스트
- 도구 호출에 MCP 클라이언트 사용

이제 배포된 MCP 서버를 대규모 AI 애플리케이션 및 워크플로에 통합할 수 있습니다!